In [1]:
# Test the several architecture of ProtoNN versus the K-NN algorithm

# import ProtoNN code
import protoNN as protograph
import protoNNTrainer as prototrainer
import utils as utils
# import general modules
import numpy as np
import torch
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
import gc

# need to change the dataset location
X, Y = utils.handleLetter26Dataset('../../../data_set/letter-recognition.data')

dataset = utils.CustomDatasetProtoNN(X, Y, rescaling=True, binary=False)

C:\Users\pietr\anaconda3\envs\edgeml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset input rescaled
Dataset loaded, input space: (20000, 16) , output space: (20000,)
Dataset reshaped, input space: (20000, 16) , output space: (20000, 26)


In [2]:
# Stratified k fold
k_folds = 5 # default 5 or 10
stratifiedk_fold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=1)
knn = KNeighborsClassifier(n_neighbors = 13)
print("Stratified KFOLD INITIALIZED")

Stratified KFOLD INITIALIZED


In [3]:
# fixed ProtoNN HYPERPARAMETERS
# regularizer of W, B and Z: HYPERPARAMETER
regW, regB, regZ = 0, 0, 0

# sparsity of W, B and Z: HYPERPARAMETER, CONSTRAINTS OF MEMORY
sparcityW, sparcityB, sparcityZ = 1, 1, 1

# initial learning rate of the OPTIMIZER: HYPERPARAMETER
learningRate = 0.1
# lossType: 'l2'
lossType= 'l2'

# TRAINING PHASE
# batchsize: HAS TO BE DEFINED
batchsize = 100
# epochs: HAS TO BE DEFINED
epochs = 100
# printStep: HAS TO BE DEFINED
printStep = 100
# valStep: HAS TO BE DEFINED
valStep = 100

# print virables of ProtoNN training
print("\nbatch size:\t", batchsize,
      "\nepochs:\t\t", epochs,
      "\nprint step:\t", printStep,
      "\nevaluation step:", valStep)


batch size:	 100 
epochs:		 100 
print step:	 100 
evaluation step: 100


In [4]:
# FOR 2 KB
size = [2]
size_parameters = [[12, 26]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 0.1, 1, 0.1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 26 
porjection dim:	 12
Model size: 1.9424000000000001 KB
Accuracy for fold 0 with m = 26.000000,d = 12: 66 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 26 
porjection dim:	 12
Model size: 1.9424000000000001 KB
Accuracy for fold 1 with m = 26.000000,d = 12: 64 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 26 
porjection dim:	 12
Model size: 1.9424000000000001 KB
Accuracy for fold 2 with m = 26.000000,d = 12: 64 %
--------------------------------
FOLD 3
--------------------------------
num prototypes:	 26 
porjection dim:	 12
Model size: 1.9424000000000001 KB
Accuracy for fold 3 with m = 26.000000,d = 12: 68 %
--------------------------------
FOLD 4
--------------------------------
num prototypes:	 26 
porjection dim:	 12
Model size: 1.9424000000000001 KB
Accuracy for fold 4 with m = 26.000000,d = 12: 68 %


In [5]:
# FOR 2 KB - sparsity on all parameters:1
size = [2]
size_parameters = [[1, 17], [5, 13]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 17 
porjection dim:	 1
Model size: 1.9 KB
Accuracy for fold 0 with m = 17.000000,d = 1: 6 %
num prototypes:	 13 
porjection dim:	 5
Model size: 1.932 KB
Accuracy for fold 0 with m = 13.000000,d = 5: 42 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 17 
porjection dim:	 1
Model size: 1.9 KB
Accuracy for fold 1 with m = 17.000000,d = 1: 4 %
num prototypes:	 13 
porjection dim:	 5
Model size: 1.932 KB
Accuracy for fold 1 with m = 13.000000,d = 5: 42 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 17 
porjection dim:	 1
Model size: 1.9 KB
Accuracy for fold 2 with m = 17.000000,d = 1: 14 %
num prototypes:	 13 
porjection dim:	 5
Model size: 1.932 KB
Accuracy for fold 2 with m = 13.000000,d = 5: 42 %
--------------------------------
FOLD 3
--------------------------------
num prototypes:	 17 
porjection dim:	 1
Mode

In [6]:
# FOR 4 KB
size = [4]
size_parameters = [[12, 55]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 0.1, 1, 0.1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 55 
porjection dim:	 12
Model size: 3.9375999999999998 KB
Accuracy for fold 0 with m = 55.000000,d = 12: 72 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 55 
porjection dim:	 12
Model size: 3.9375999999999998 KB
Accuracy for fold 1 with m = 55.000000,d = 12: 73 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 55 
porjection dim:	 12
Model size: 3.9375999999999998 KB
Accuracy for fold 2 with m = 55.000000,d = 12: 73 %
--------------------------------
FOLD 3
--------------------------------
num prototypes:	 55 
porjection dim:	 12
Model size: 3.9375999999999998 KB
Accuracy for fold 3 with m = 55.000000,d = 12: 75 %
--------------------------------
FOLD 4
--------------------------------
num prototypes:	 55 
porjection dim:	 12
Model size: 3.9375999999999998 KB
Accuracy for fold 4 with m = 55.000000,d = 12: 77 %


In [7]:
# FOR 4 KB - sparsity on all parameters:1
size = [4]
size_parameters = [[7, 26]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 26 
porjection dim:	 7
Model size: 3.88 KB
Accuracy for fold 0 with m = 26.000000,d = 7: 71 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 26 
porjection dim:	 7
Model size: 3.88 KB
Accuracy for fold 1 with m = 26.000000,d = 7: 72 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 26 
porjection dim:	 7
Model size: 3.88 KB
Accuracy for fold 2 with m = 26.000000,d = 7: 72 %
--------------------------------
FOLD 3
--------------------------------
num prototypes:	 26 
porjection dim:	 7
Model size: 3.88 KB
Accuracy for fold 3 with m = 26.000000,d = 7: 68 %
--------------------------------
FOLD 4
--------------------------------
num prototypes:	 26 
porjection dim:	 7
Model size: 3.88 KB
Accuracy for fold 4 with m = 26.000000,d = 7: 74 %
--------------------------------
K-FOLD CROSS VALIDATION RESULTS FOR 5
---------

In [8]:
# FOR 8 KB
size = [8]
size_parameters = [[15, 87]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z: HAS TO BE DEFINED, CONSTRAINTS OF MEMORY
        sparcityW, sparcityB, sparcityZ = 0.5, 1, 0.1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 87 
porjection dim:	 15
Model size: 7.9896 KB
Accuracy for fold 0 with m = 87.000000,d = 15: 85 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 87 
porjection dim:	 15
Model size: 7.9896 KB
Accuracy for fold 1 with m = 87.000000,d = 15: 85 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 87 
porjection dim:	 15
Model size: 7.9896 KB
Accuracy for fold 2 with m = 87.000000,d = 15: 85 %
--------------------------------
FOLD 3
--------------------------------
num prototypes:	 87 
porjection dim:	 15
Model size: 7.9896 KB
Accuracy for fold 3 with m = 87.000000,d = 15: 85 %
--------------------------------
FOLD 4
--------------------------------
num prototypes:	 87 
porjection dim:	 15
Model size: 7.9896 KB
Accuracy for fold 4 with m = 87.000000,d = 15: 87 %
--------------------------------
K-FOLD CROSS VALIDATION RES

In [9]:
# FOR 8 KB - sparsity on all parameters:1
size = [8]
size_parameters = [[10, 50], [9, 52]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 50 
porjection dim:	 10
Model size: 7.84 KB
Accuracy for fold 0 with m = 50.000000,d = 10: 81 %
num prototypes:	 52 
porjection dim:	 9
Model size: 7.856 KB
Accuracy for fold 0 with m = 52.000000,d = 9: 81 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 50 
porjection dim:	 10
Model size: 7.84 KB
Accuracy for fold 1 with m = 50.000000,d = 10: 81 %
num prototypes:	 52 
porjection dim:	 9
Model size: 7.856 KB
Accuracy for fold 1 with m = 52.000000,d = 9: 79 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 50 
porjection dim:	 10
Model size: 7.84 KB
Accuracy for fold 2 with m = 50.000000,d = 10: 80 %
num prototypes:	 52 
porjection dim:	 9
Model size: 7.856 KB
Accuracy for fold 2 with m = 52.000000,d = 9: 79 %
--------------------------------
FOLD 3
--------------------------------
num prototypes:	 50 
porjection d

In [10]:
# FOR 16 KB
size = [16]
size_parameters = [[15, 186]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 0.5, 1, 0.1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 186 
porjection dim:	 15
Model size: 15.9888 KB
Accuracy for fold 0 with m = 186.000000,d = 15: 88 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 186 
porjection dim:	 15
Model size: 15.9888 KB
Accuracy for fold 1 with m = 186.000000,d = 15: 88 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 186 
porjection dim:	 15
Model size: 15.9888 KB
Accuracy for fold 2 with m = 186.000000,d = 15: 87 %
--------------------------------
FOLD 3
--------------------------------
num prototypes:	 186 
porjection dim:	 15
Model size: 15.9888 KB
Accuracy for fold 3 with m = 186.000000,d = 15: 89 %
--------------------------------
FOLD 4
--------------------------------
num prototypes:	 186 
porjection dim:	 15
Model size: 15.9888 KB
Accuracy for fold 4 with m = 186.000000,d = 15: 90 %
--------------------------------
K-FOLD CROSS

In [11]:
# FOR 16 KB - sparsity on all parameters:1
size = [16]
size_parameters = [[20, 78], [15, 90], [10, 104]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 78 
porjection dim:	 20
Model size: 15.632 KB
Accuracy for fold 0 with m = 78.000000,d = 20: 86 %
num prototypes:	 90 
porjection dim:	 15
Model size: 15.72 KB
Accuracy for fold 0 with m = 90.000000,d = 15: 87 %
num prototypes:	 104 
porjection dim:	 10
Model size: 15.616 KB
Accuracy for fold 0 with m = 104.000000,d = 10: 85 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 78 
porjection dim:	 20
Model size: 15.632 KB
Accuracy for fold 1 with m = 78.000000,d = 20: 86 %
num prototypes:	 90 
porjection dim:	 15
Model size: 15.72 KB
Accuracy for fold 1 with m = 90.000000,d = 15: 86 %
num prototypes:	 104 
porjection dim:	 10
Model size: 15.616 KB
Accuracy for fold 1 with m = 104.000000,d = 10: 86 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 78 
porjection dim:	 20
Model size: 15.632 KB
Accuracy for fold 2 with m

In [15]:
# FOR 32 KB - sparsity on all parameters:1
size = [32]
size_parameters = [[31, 130], [15, 182], [15, 130], [10, 208]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 130 
porjection dim:	 31
Model size: 31.624 KB
Accuracy for fold 0 with m = 130.000000,d = 31: 90 %
num prototypes:	 182 
porjection dim:	 15
Model size: 30.808 KB
Accuracy for fold 0 with m = 182.000000,d = 15: 89 %
num prototypes:	 130 
porjection dim:	 15
Model size: 22.28 KB
Accuracy for fold 0 with m = 130.000000,d = 15: 88 %
num prototypes:	 208 
porjection dim:	 10
Model size: 30.592 KB
Accuracy for fold 0 with m = 208.000000,d = 10: 87 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 130 
porjection dim:	 31
Model size: 31.624 KB
Accuracy for fold 1 with m = 130.000000,d = 31: 90 %
num prototypes:	 182 
porjection dim:	 15
Model size: 30.808 KB
Accuracy for fold 1 with m = 182.000000,d = 15: 89 %
num prototypes:	 130 
porjection dim:	 15
Model size: 22.28 KB
Accuracy for fold 1 with m = 130.000000,d = 15: 88 %
num prototypes:	 208 
porjection dim:

In [16]:
# FOR 64 KB - sparsity on all parameters:1
size = [64]
size_parameters = [[33, 260], [15, 380], [10,  440], [15, 260]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 260 
porjection dim:	 33
Model size: 63.472 KB
Accuracy for fold 0 with m = 260.000000,d = 33: 90 %
num prototypes:	 380 
porjection dim:	 15
Model size: 63.28 KB
Accuracy for fold 0 with m = 380.000000,d = 15: 86 %
num prototypes:	 440 
porjection dim:	 10
Model size: 64.0 KB
Accuracy for fold 0 with m = 440.000000,d = 10: 80 %
num prototypes:	 260 
porjection dim:	 15
Model size: 43.6 KB
Accuracy for fold 0 with m = 260.000000,d = 15: 91 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 260 
porjection dim:	 33
Model size: 63.472 KB
Accuracy for fold 1 with m = 260.000000,d = 33: 89 %
num prototypes:	 380 
porjection dim:	 15
Model size: 63.28 KB
Accuracy for fold 1 with m = 380.000000,d = 15: 85 %
num prototypes:	 440 
porjection dim:	 10
Model size: 64.0 KB
Accuracy for fold 1 with m = 440.000000,d = 10: 82 %
num prototypes:	 260 
porjection dim:	 15
M

In [17]:
knn = KNeighborsClassifier(n_neighbors = 5)
# Start print
knn_results = {}
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # K-NN
    # test KNN
    knn.fit(train_inputs, train_targets)
    knn_foldscore = knn.score(test_inputs,test_targets)
    knn_results[fold] = knn_foldscore*100
    print("KNN accuracy:",knn_foldscore)
    print('--------------------------------')

sum = 0.0
for key, value in knn_results.items():
    print(f'Fold {key}: {value} %')
    sum += value
print(f'Average KNN: {sum/len(knn_results.items())} %')

--------------------------------
FOLD 0
--------------------------------
KNN accuracy: 0.92575
--------------------------------
FOLD 1
--------------------------------
KNN accuracy: 0.9315
--------------------------------
FOLD 2
--------------------------------
KNN accuracy: 0.92475
--------------------------------
FOLD 3
--------------------------------
KNN accuracy: 0.927
--------------------------------
FOLD 4
--------------------------------
KNN accuracy: 0.93
--------------------------------
Fold 0: 92.575 %
Fold 1: 93.15 %
Fold 2: 92.475 %
Fold 3: 92.7 %
Fold 4: 93.0 %
Average KNN: 92.78 %


In [19]:
knn = KNeighborsClassifier(n_neighbors = 3)
# Start print
knn_results = {}
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # K-NN
    # test KNN
    knn.fit(train_inputs, train_targets)
    knn_foldscore = knn.score(test_inputs,test_targets)
    knn_results[fold] = knn_foldscore*100
    print("KNN accuracy:",knn_foldscore)
    print('--------------------------------')

sum = 0.0
for key, value in knn_results.items():
    print(f'Fold {key}: {value} %')
    sum += value
print(f'Average KNN: {sum/len(knn_results.items())} %')

--------------------------------
FOLD 0
--------------------------------
KNN accuracy: 0.94125
--------------------------------
FOLD 1
--------------------------------
KNN accuracy: 0.9425
--------------------------------
FOLD 2
--------------------------------
KNN accuracy: 0.936
--------------------------------
FOLD 3
--------------------------------
KNN accuracy: 0.9415
--------------------------------
FOLD 4
--------------------------------
KNN accuracy: 0.94275
--------------------------------
Fold 0: 94.125 %
Fold 1: 94.25 %
Fold 2: 93.60000000000001 %
Fold 3: 94.15 %
Fold 4: 94.27499999999999 %
Average KNN: 94.08 %


In [18]:
import winsound
# to notify that I'm done running
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)
winsound.Beep(500, duration)